In [52]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import unicodedata
import string
import torch
import torch.nn as nn
import os

In [53]:
print(torch.__version__)

0.4.0


In [54]:
all_letters = string.ascii_letters + " .,;'-"
n_letters = len(all_letters) + 1 # Plus EOS marker

In [55]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

In [56]:
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

In [57]:
def findFiles(path): return glob.glob(path)

In [72]:
category_lines = {}
all_categories = []
all_categoryi = []
for filename in findFiles('data/*.txt'):
    category = filename.split('\\')[-1].split('.')[0]
#     categoryi = os.path.splitext(os.path.basename(filename))[0]
    if category == categoryi:
        print('Yes')
    else:
        print('No')
    all_categories.append(category)
    all_categoryi.append(categoryi)
    category_lines[category] = readLines(filename)
n_categories = len(all_categories)
if n_categories == 0:
    raise RuntimeError('Data not found. Make sure that you downloaded data '
        'from https://download.pytorch.org/tutorial/data.zip and extract it to '
        'the current directory.')

print('# categories:', n_categories, all_categories)
print('# categories:', n_categories, all_categoryi)
print(unicodeToAscii("O'Néàl"))

No
No
No
No
No
No
No
No
No
No
No
No
No
No
No
No
No
Yes
# categories: 18 ['Arabic', 'Chinese', 'Czech', 'Dutch', 'English', 'French', 'German', 'Greek', 'Irish', 'Italian', 'Japanese', 'Korean', 'Polish', 'Portuguese', 'Russian', 'Scottish', 'Spanish', 'Vietnamese']
# categories: 18 ['Vietnamese', 'Vietnamese', 'Vietnamese', 'Vietnamese', 'Vietnamese', 'Vietnamese', 'Vietnamese', 'Vietnamese', 'Vietnamese', 'Vietnamese', 'Vietnamese', 'Vietnamese', 'Vietnamese', 'Vietnamese', 'Vietnamese', 'Vietnamese', 'Vietnamese', 'Vietnamese']
O'Neal


In [73]:
class RNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size)
        self.o2o = nn.Linear(output_size + hidden_size, output_size)
        self.drop_out = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input, hidden), dim = 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((output, hidden), dim =1)
        output = self.o2o(output_combined)
        output = self.drop_out(output)
        output = self.softmax(output)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [74]:
import random

# Random item from a list
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

# Get a random category and random line from that category
def randomTrainingPair():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    return category, line

In [75]:
def categotyTensor(category):
    li = all_categories.index(category)
    tensor = torch.zeros(1, n_categories)
    tensor[0][li] = 1
    return tensor

def inputTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

def targetTensor(line):
    letter_indices = [all_letters.find(line[li]) for li in range(len(line))]
    letter_indices.append(n_letters - 1)
    return torch.LongTensor(letter_indices)

In [76]:
def randomTrainingExample(): 
    catoegory, line = randomTrainingPair()
    category_tensor = categotyTensor(category)
    input_tensor = inputTensor(line)
    target_tensor = targetTensor(line)
    return category_tensor, input_tensor, target_tensor

In [77]:
crieterion = nn.NLLLoss()
learning_rate = 0.001

In [78]:
def train(category_tensor, input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden  = rnn.initHidden()
    rnn.zero_grad()
    loss = 0
#     print(type(loss))
    #Forwarding the input through time
    for i in range(input_line_tensor.size(0)):
        output, hidden = rnn(category_tensor, input_line_tensor[i], hidden)
        l = crieterion(output, target_line_tensor[i])
#         print(type(loss))
        loss += l
#         print(type(loss))
    loss.backward()
    
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)
    
    return output, loss.item() / input_line_tensor.size(0)

In [79]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [ ]:
rnn = RNN(n_letters, 128, n_letters)

n_iters = 100000
print_every = 5000
plot_every = 500
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time()

for iter in range(1, n_iters + 1):
#     print(iter)
    output, los = train(*randomTrainingExample())
    total_loss += los
#     print(total_loss)

    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, los))

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0

1m 28s (5000 5%) 1.5607
2m 55s (10000 10%) 1.2310
